In [24]:
import pandas as pd
import numpy as np
from jupyter_utils import jupyter_setup, load_tracker
import ipywidgets as widgets
jupyter_setup()
import plotly.graph_objects as go
import plotly.express as ex
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

D:\Archiv\Studium\Master\6.-Semester\Masters_Thesis\Git\acoustic_covid_detection\python


In [25]:
tracker = load_tracker()

Loading Tracker from: D:/Archiv/Studium/Master/6.-Semester/Masters_Thesis/Git/acoustic_covid_detection/python/run/tracker_saves/2023-02-08_brogrammers_15_mfccs_trackerclasstest actual hyperparameter run.pickle

Parameters used for finding the best performing epoch:
{'metric_used': 'auc_roc', 'smoothing': 5, 'ignore_first_n_epochs': 5}

Tracker contains:
>>  24 <<  parameter runs
>>  5 <<  folds per run
>> 75 <<  epochs


In [26]:
tracker.compute_overall_metrics(smooth_n_samples=1, ignore_first_n_epochs=10, metric_used_for_performance_analysis="auc_roc")


Parameters used for finding the best performing epoch:
{'metric_used': 'auc_roc', 'smoothing': 1, 'ignore_first_n_epochs': 10}


In [27]:
tracker.style_metric_performance_df(tracker.compact_metric_performance_df, sort_by="f1_score")

,auc_roc,loss,accuracy,f1_score,auc_prec_recall,precision,tpr,tnr,batch,lr,wd,sigma,shift,lr_decay
9,0.807572,0.457864,0.816418,0.556531,0.572607,0.580420,0.538100,0.893780,32,0.001,0.001,0,True,0.9
1,0.818657,0.425928,0.820925,0.550872,0.556079,0.550520,0.552800,0.887280,32,0.0001,0.001,0,True,0.9
6,0.810674,0.434234,0.816613,0.542423,0.550040,0.557600,0.529560,0.890560,32,0.0005,0.0001,0,True,0.85
8,0.802928,0.441405,0.821468,0.540873,0.539394,0.568800,0.520400,0.897920,32,0.001,0.001,0,True,0.85
5,0.813202,0.435845,0.816984,0.539594,0.571299,0.570120,0.523120,0.893320,32,0.0005,0.001,0,True,0.9
17,0.805157,0.435120,0.820003,0.535277,0.549494,0.575520,0.502420,0.901700,128,0.0005,0.001,0,True,0.9
21,0.799157,0.441320,0.823325,0.534813,0.544372,0.567720,0.511920,0.902360,128,0.001,0.001,0,True,0.9
20,0.802048,0.426496,0.820948,0.528992,0.564173,0.567140,0.497320,0.903580,128,0.001,0.001,0,True,0.85
13,0.792893,0.441559,0.809963,0.519139,0.525478,0.543240,0.502000,0.890580,128,0.0001,0.001,0,True,0.9
3,0.811132,0.426931,0.810461,0.518856,0.544149,0.531360,0.508320,0.886780,32,0.0001,0.0001,0,True,0.9


In [28]:
tracker.style_metric_performance_df(tracker.full_metric_performance_df, sort_by="auc_roc")

,auc_roc,auc_roc-σ,loss,loss-σ,accuracy,accuracy-σ,f1_score,f1_score-σ,auc_prec_recall,auc_prec_recall-σ,precision,precision-σ,tpr_at_95,tpr_at_95-σ,tpr,tpr-σ,tnr,tnr-σ,batch,lr,wd,sigma,shift,lr_decay
1,0.818657,0.021143,0.425928,0.032061,0.820925,0.018650,0.550872,0.022462,0.556079,0.049744,0.550520,0.022461,0.367895,0.056866,0.552800,0.037165,0.887280,0.018965,32,0.0001,0.001,0,True,0.9
5,0.813202,0.007338,0.435845,0.023931,0.816984,0.013888,0.539594,0.014291,0.571299,0.033042,0.570120,0.065079,0.355753,0.049393,0.523120,0.056452,0.893320,0.031152,32,0.0005,0.001,0,True,0.9
3,0.811132,0.009742,0.426931,0.012028,0.810461,0.012412,0.518856,0.030013,0.544149,0.044136,0.531360,0.042313,0.329048,0.045007,0.508320,0.030154,0.886780,0.011719,32,0.0001,0.0001,0,True,0.9
6,0.810674,0.010954,0.434234,0.019616,0.816613,0.019409,0.542423,0.022581,0.550040,0.039126,0.557600,0.017717,0.343144,0.038406,0.529560,0.038736,0.890560,0.016945,32,0.0005,0.0001,0,True,0.85
15,0.808366,0.014925,0.430313,0.014143,0.809774,0.012416,0.500360,0.036853,0.532504,0.046478,0.537480,0.044982,0.291641,0.077771,0.470880,0.045783,0.895900,0.018646,128,0.0001,0.0001,0,True,0.9
9,0.807572,0.009471,0.457864,0.025688,0.816418,0.013793,0.556531,0.015728,0.572607,0.047199,0.580420,0.040455,0.342231,0.064875,0.538100,0.031657,0.893780,0.011150,32,0.001,0.001,0,True,0.9
2,0.806746,0.014702,0.430133,0.014826,0.810426,0.010927,0.513881,0.036919,0.550297,0.042525,0.553740,0.032043,0.311604,0.048840,0.480120,0.043258,0.897600,0.009408,32,0.0001,0.0001,0,True,0.85
10,0.806208,0.007905,0.421797,0.013761,0.818033,0.014795,0.503652,0.024696,0.545477,0.040573,0.546780,0.024688,0.338321,0.047712,0.471560,0.053158,0.903760,0.015483,32,0.001,0.0001,0,True,0.85
17,0.805157,0.013554,0.435120,0.029856,0.820003,0.020438,0.535277,0.020691,0.549494,0.043464,0.575520,0.033886,0.366452,0.080412,0.502420,0.030656,0.901700,0.023942,128,0.0005,0.001,0,True,0.9
19,0.803403,0.009186,0.438325,0.019220,0.813963,0.006341,0.515280,0.025707,0.530512,0.040554,0.540540,0.016375,0.311075,0.038936,0.493680,0.041477,0.894420,0.009605,128,0.0005,0.0001,0,True,0.9


In [ ]:
_ = widgets.interact(tracker.boxplot_run_statistics, metric=tracker.metrics_used)

In [17]:
_ = widgets.interact(tracker.show_all_runs, 
                 mode=["eval", "train"], 
                 metric=tracker.metrics_used, 
                 n_samples_for_smoothing=widgets.IntSlider(value=tracker.performance_eval_params["smoothing"], min=1, max=10), 
                 show_separate_folds=False, 
                 show_std_area_plot=True)

interactive(children=(Dropdown(description='mode', options=('eval', 'train'), value='eval'), Dropdown(descript…

In [18]:
_ = widgets.interact(tracker.show_single_run, 
                     
                 run_id=tracker.run_ids,
                 mode=["eval", "train"], 
                 metric=tracker.metrics_used, 
                 n_samples_for_smoothing=widgets.IntSlider(value=tracker.performance_eval_params["smoothing"], min=1, max=10), 
                 show_separate_folds=True, 
                 show_std_area_plot=True)

interactive(children=(Dropdown(description='run_id', options=("{'batch': '32', 'lr': '0.0005', 'wd': '0.001', …